# Amazon SageMaker Multi-Model Endpoints using MONAI Application Package container

With [Amazon SageMaker Multi-Model Endpoints (MME)](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html), customers can create an endpoint that seamlessly hosts up to thousands of models, which can be served from a common inference container. It provides a scalable and cost-effective way to deploy large number of deep learning models. MME will run multiple models on a GPU core, share GPU instances behind an endpoint across multiple models and dynamically load/unload models based on the incoming traffic. With this, customers can significantly save cost and achieve best price performance. At a high level, Amazon SageMaker manages the loading and unloading of models for a multi-model endpoint, as they are needed. When an invocation request is made for a particular model, Amazon SageMaker routes the request to an instance assigned to that model, downloads the model artifacts from S3 onto that instance, and initiates loading of the model into the memory of the container. As soon as the loading is complete, Amazon SageMaker performs the requested invocation and returns the result. If the model is already loaded in memory on the selected instance, the downloading and loading steps are skipped and the invocation is performed immediately. For the inference container to serve multiple models in a multi-model endpoint, it must implement [additional APIs](https://docs.aws.amazon.com/sagemaker/latest/dg/build-multi-model-build-container.html) in order to load, list, get, unload and invoke specific models. [Example notebook](https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/multi_model_bring_your_own) is available for more implementation details.

Let's start by creating a SageMaker session and specifying:
- The S3 bucket and prefix that you want to use for the model. 
- The IAM role arn used to give hosting access to your data.

In [ ]:
import boto3
import os
from botocore.exceptions import ClientError
from sagemaker import get_execution_role

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

bucket = "sagemaker-{}-{}".format(region, account_id)
prefix= "medicalimaging-mms-map"

role = get_execution_role()

%store -r

### 1. Create Inference Container

We will use CI/CD pipeline to build the inference container with MONAI Application Package (MAP). The steps here followed the details in [this notebook](https://github.com/aws-samples/aws-research-workshops/blob/mainline/notebooks/container/container-cicd.ipynb).

#### 1.0 Create IAM roles and ECR repo

In [ ]:
### We start by setting up the proper access permissions using the IAM service. Each service (CodePipeline, CodeBuild) needs its own policies. We also need to allow these services to access other related services on our behalf (S3 and ECR).
iam_client = boto3.client('iam')

def create_service_role_with_policies(role_name, service_name, policy_arns):
    try:
        resp = iam_client.create_role(RoleName=role_name,
                                 AssumeRolePolicyDocument='{"Version":"2012-10-17","Statement":[{"Sid":"","Effect":"Allow","Principal":{"Service": "' + service_name+'"},"Action":"sts:AssumeRole"}]}')
        for policy in policy_arns:
            resp = iam_client.attach_role_policy(PolicyArn=policy,RoleName=role_name)
    except ClientError  as e:
        if e.response['Error']['Code'] == 'EntityAlreadyExists':
            print(f"{role_name} already exists, ignore")
        else: 
            raise  e
    
    resp = iam_client.get_role(RoleName=role_name)
    return resp['Role']['Arn']


codepipeline_service_role_name = f"{prefix}-codepipeline-service-role"
codepipeline_policies = ['arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess', 
                         'arn:aws:iam::aws:policy/AWSCodeCommitFullAccess',
                         'arn:aws:iam::aws:policy/AmazonS3FullAccess',
                         'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess'
                        ]
codepipeline_role_arn = create_service_role_with_policies(codepipeline_service_role_name, 'codepipeline.amazonaws.com', codepipeline_policies )
print(f"code pipeline IAM role ARN: {codepipeline_role_arn}")
              
codebuild_service_role_name = f"{prefix}-codebuild-service-role"
codebuild_policies = ['arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess',
                      'arn:aws:iam::aws:policy/CloudWatchFullAccess',
                      'arn:aws:iam::aws:policy/AmazonS3FullAccess',
                      'arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess']
codebuild_role_arn = create_service_role_with_policies(codebuild_service_role_name, 'codebuild.amazonaws.com', codebuild_policies )
print(f"code build IAM role ARN: {codebuild_role_arn}")


### Before we can actually build our image, we need to have the repository referenced in our (post_build) phase. We will use boto3 again to interact with the AWS ECR APIs. We will actually use the repository after the container image is built.
ecr_client = boto3.client(service_name="ecr")
fullname=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest"
try:
    res=ecr_client.describe_repositories(
        repositoryNames=[
            prefix
        ]
    )
    print(f"ECR repository ARN: {res['repositories'][0]['repositoryArn']}")
except:
    print('Create new repository...')
    res=ecr_client.create_repository(
        repositoryName=prefix
    )
    print(f"new ECR repository ARN: {res['repository']['repositoryArn']}")

#### 1.1 Create an AWS CodeCommit repo and checkin the files

In [ ]:
# prepare the files for the checkin. 
with open(f"{os.getcwd()}/src/Dockerfile", "r") as f:
    dockerfile_content = f.read()
    
with open(f"{os.getcwd()}/src/model_handler.py", "r") as f:
    model_handler_content = f.read()
    
with open(f"{os.getcwd()}/src/dockerd-entrypoint.py", "r") as f:
    entrypoint_content = f.read()

with open(f"{os.getcwd()}/src/buildspec.yml", "r") as f:
    buildspec_content = f.read()
    
with open(f"{os.getcwd()}/src/code/app.py", "r") as f:
    app_content = f.read()
    
with open(f"{os.getcwd()}/src/code/ahi_data_loader_operator.py", "r") as f:
    dataloader_content = f.read()
    
put_files=[
    {
        'filePath': 'Dockerfile',
        'fileContent': dockerfile_content
    },
    {
        'filePath': 'model_handler.py',
        'fileContent': model_handler_content
    },
    {
        'filePath': 'dockerd-entrypoint.py',
        'fileContent': entrypoint_content
    },
    {
        'filePath': 'buildspec.yml',
        'fileContent': buildspec_content
    },
    {
        'filePath': 'app.py',
        'fileContent': app_content
    },
    {
        'filePath': 'ahi_data_loader_operator.py',
        'fileContent': dataloader_content
    }
]

codecommit_client = boto3.client('codecommit')
codecommit_name= f"Source-{prefix}"

try:
    resp = codecommit_client.create_repository(repositoryName=codecommit_name)
    print(f"new code repository: {res}")
except ClientError as e:
    if e.response['Error']['Code'] == 'RepositoryNameExistsException':
        print(f"Repo {prefix} exists, use that one")

try:
    resp = codecommit_client.get_branch(repositoryName=codecommit_name, branchName='main')
    parent_commit_id = resp['branch']['commitId']
except ClientError as e:
    if e.response['Error']['Code'] == 'BranchDoesNotExistException':
        # the repo is new, create it 
        codecommit_client.create_commit(repositoryName=codecommit_name, branchName="main", putFiles=put_files)
    else:
        try:
            resp = codecommit_client.create_commit(repositoryName=codecommit_name, branchName="main", parentCommitId=parent_commit_id, putFiles=put_files) 
        except ClientError as ee:
            if ee.response['Error']['Code'] == 'NoChangeException':
                print('No change detected. skip commit')

#### 1.2 Create a CodeBuild project and CodePipeline to build container image
We use an instance managed by AWS (see computeType below) to build the container using a standard Amazon Linux 2 build environment. The CodeBuild process is triggered by the CodeCommit code checkins.
Note: codebuild-service-role takes a little longer to propagate. If you see a permission error, please retry again in a minute.

In [ ]:
codebuild_client = boto3.client('codebuild')
codebuild_name = f"Build-{prefix}" 

codepipeline_client = boto3.client('codepipeline')
codepipeline_name = f"Pipeline-{prefix}" 

try: 
    resp = codebuild_client.create_project(
        name=codebuild_name, 
        description="MMS Monai Deploy build demo",
        source= {
            'type': "CODEPIPELINE"
        },
        artifacts= {
            "type": "CODEPIPELINE",
            "name": codepipeline_name
        },
        environment= {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "AWS_DEFULT_REGION",
                    "value": region,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_ACCOUNT_ID",
                    "value": account_id,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": prefix,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_TAG",
                    "value": "dev",
                    "type": "PLAINTEXT"
                }
            ],
            "privilegedMode": True,
            "imagePullCredentialsType": "CODEBUILD"               
        },
        logsConfig= {
            "cloudWatchLogs": {
                "status": "ENABLED",
                "groupName": f"log-{prefix}"
            },
            "s3Logs": {
                "status": "DISABLED"
            }
        },
        serviceRole= codebuild_role_arn
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
        print(f"CodeBuild project {codebuild_name} exists, skip...")
    else:
        raise e
        
print(f"CodeBuild project name {codebuild_name}")

In [ ]:
### create a pipeline with two stages commit and build.
stage1 = {
    "name":f"{codecommit_name}",
    "actions": [
        {
            "name": "Source",
            "actionTypeId": {
                "category": "Source",
                "owner": "AWS",
                "provider": "CodeCommit",
                "version": "1"
            },
            "runOrder": 1,
            "configuration": {
                "BranchName": "main",
                "OutputArtifactFormat": "CODE_ZIP",
                "PollForSourceChanges": "true",
                "RepositoryName": codecommit_name
            },
            "outputArtifacts": [
                {
                    "name": "SourceArtifact"
                }
            ],
            "inputArtifacts": [],
            "region": region,
            "namespace": "SourceVariables"
        }
    ]
}

stage2 = {
   "name": f"{codebuild_name}",
    "actions": [
        {
            "name": "Build",
            "actionTypeId": {
                "category": "Build",
                "owner": "AWS",
                "provider": "CodeBuild",
                "version": "1"
            },
            "runOrder": 1,
            "configuration": {
                "ProjectName": codebuild_name
            },
            "outputArtifacts": [
                {
                    "name": "BuildArtifact"
                }
            ],
            "inputArtifacts": [
                {
                    "name": "SourceArtifact"
                }
            ],
            "region": region,
            "namespace": "BuildVariables"
        }
    ]    
}


stages = [ stage1, stage2]


pipeline = {
    'name': codepipeline_name,
    'roleArn': codepipeline_role_arn,
    'artifactStore': {
        'type': 'S3',
        'location': bucket
    }, 
    'stages': stages
}

try:
    resp = codepipeline_client.create_pipeline( pipeline= pipeline)
    print("Created pipeline",resp)
except ClientError as e:
    if e.response['Error']['Code'] == 'PipelineNameInUseException':
       print(f"Codepipeline {codepipeline_name} already exists " )
    

Building the container may take ~20 minutes, you can check if container is ready to use below

In [ ]:
from IPython.display import display, clear_output
import time
while True:
    resp = ecr_client.describe_images(repositoryName=prefix)
    if resp['imageDetails']:
        for image in resp['imageDetails']:
            print("image pushed at: " + str(image['imagePushedAt']))
        break
    else:
        clear_output(wait=True)
        display("Build not done yet, please wait and retry this step. Please do not proceed until you see the 'image pushed' message")
        time.sleep(20)
# this is used later in job_definition for AWS Batch
image_uri= f"{account_id}.dkr.ecr.{region}.amazonaws.com/{prefix}:dev"
print(image_uri)

### 2. Create a multi-model inference endpoint
#### 2.1 Create Model

We will use SageMaker boto3 client [Create Model API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateModel.html) to create the Model entity for multi-model endpoints, with the container definition, ModelName, and ExecutionRoleArn.

In the container definition, define the ModelDataUrl to specify the S3 directory that contains all the models that SageMaker multi-model endpoint will use to load and serve predictions. Set `Mode` to `MultiModel` to indicates SageMaker would create the endpoint with MME container specifications. We set the container with an image that supports deploying multi-model endpoints with GPU. The container's `ModelDataUrl` is the S3 prefix where the model artifacts that are invokable by the endpoint are located. The rest of the S3 path will be specified when invoking the model.


In [ ]:
from time import gmtime, strftime

### upload model to s3
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)
print(os.path.join(bucket, key))

##############################################################################
model_name = "DEMO-MONAIDeployModel" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "s3://{}/{}/".format(bucket, prefix)

print("Model name: " + model_name)
print("Model data Url: " + model_url)
print("Container image: " + image_uri)

container = {"Image": image_uri, "ModelDataUrl": model_url, "Mode": "MultiModel"}

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

### Create endpoint configuration
Create a multi-model endpoint configurations using create_endpoint_config boto3 API. Specify an accelerated GPU computing instance in InstanceType, in this post we will use g4dn.xlarge instance. We recommend configuring your endpoints with at least two instances. This allows SageMaker to provide a highly available set of predictions across multiple Availability Zones for the models.

In [ ]:
endpoint_config_name = "DEMO-MONAIDeployEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ]
)

print("Endpoint config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

### Create endpoint
Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to InService once the deployment is successful.

In [ ]:
import time

endpoint_name = "DEMO-MONAIDeployEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint name: " + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

waiter = sm_client.get_waiter("endpoint_in_service")
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

## Invoke models
Once the endpoint is successfully created, we can send inference request to multi-model endpoint using invoke_enpoint API. We specify the TargetModel in the invocation call and pass in the payload for each model type. Sample invocation for PyTorch model and TensorRT model is shown below

Now we invoke the models that we uploaded to S3 previously. The first invocation of a model may be slow, since behind the scenes, SageMaker is downloading the model artifacts from S3 to the instance and loading it into the container.

In [ ]:
%%time

import json

Payload = {
    "inputs": [
        {"datastoreId": datastoreId, "imageSetId": next(iter(imageSetIds))}
    ]
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    # ContentType="application/x-image",
    ContentType="application/json",
    Accept="application/json",
    TargetModel="model.tar.gz",  # this is the rest of the S3 path where the model artifacts are located
    Body=json.dumps(Payload)
)

print(*json.loads(response["Body"].read()), sep="\n")

### 3. Delete the hosting resources

In [ ]:
codepipeline_client.delete_pipeline(name=codepipeline_name)
codebuild_client.delete_project(name=codebuild_name)
codecommit_client.delete_repository(repositoryName=codecommit_name)
ecr_client.delete_repository(repositoryName=prefix, force=True)

def delete_service_role_with_policies(role_name, policy_arns):
    iam_client = boto3.client('iam')
    try:
        for policy in policy_arns:
            try: 
                resp = iam_client.detach_role_policy(PolicyArn=policy,RoleName=role_name)
            except ClientError as ee:
                if ee.response['Error']['Code'] == 'NoSuchEntity':
                    print("Policy not attached, ignore")
                    
        resp = iam_client.delete_role(RoleName=role_name)
        print(f"deleted service role {role_name}")
    except ClientError  as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"{role_name} already deleted, ignore")
        else: 
            raise  e
            
delete_service_role_with_policies(codepipeline_service_role_name, codepipeline_policies )
delete_service_role_with_policies(codebuild_service_role_name, codebuild_policies )

sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)